In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# preprocessing
from sklearn.preprocessing import LabelEncoder
dataset['sex'] = dataset['sex'].replace(to_replace='male', value=1)
dataset['sex'] = dataset['sex'].replace(to_replace='female', value=0)
dataset['smoker'] = dataset['smoker'].replace(to_replace='no', value=0)
dataset['smoker'] = dataset['smoker'].replace(to_replace='yes', value=1)
le = LabelEncoder()
len = le.fit_transform(dataset['region'])
dataset['region'] = len
dataset

In [ ]:
from sklearn.model_selection import train_test_split
train, test_dataset = train_test_split(dataset,test_size=0.2, random_state=42)
train_labels = train['expenses']
test_labels = test_dataset['expenses']
train = train.drop(columns='expenses')
test_dataset = test_dataset.drop(columns='expenses')
train_labels
train


In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# model = LinearRegression()
# model.fit(train, train_labels)
# result = model.predict(test_dataset)
# print(result)

# print(np.mean(np.abs((test_labels-result)/test_labels))*100)

input_shape = train.keys().size

model = keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)

])
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae', 'mse'],
)
model.fit(train,train_labels, epochs=100)
model.summary()


In [ ]:

res = model.evaluate(test_dataset, test_labels, verbose=2)
res

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset,test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
